In [1]:
using BenchmarkTools

# Custom linear algebra

In [2]:
"""
    CpBDBt!(C, B, d)

    Compute C += B*D*B'
        where B' is the transpose of B, and D = diag(d1, ..., dn)
        and B is mxn dense, C is mxm (dense), d is nx1 (dense).

    C is modified in-place (no memory allocated),
    only its upper-triangular part is modified.
"""
function CpBDBt!(C::Array{Float64,2}, B::Array{Float64, 2}, d::Array{Float64, 1})
    
    m, k = size(B)
    
    # TODO: remove @assert macro, and throw error instead
    @assert (m, m) == size(C)
    @assert (k,) == size(d)
    
    # compute symmetric rank-k update
    # only the upper-triangular part of C is modified
    temp::Float64 = 0.0
    for j=1:m
        for l = 1:k
            @inbounds temp = d[l] * B[j, l]
            if temp == 0.0
                # skip loop if temp is zero
                continue 
            end
            for i=1:j
                @inbounds C[i, j] = C[i, j] + temp * B[i, l]
            end
        end
    end
    
    return nothing
end

CpBDBt!

# Data structure

## Primal-block angular matrices

$A$ is the constraint matrix of the RMP, of shape $m \times n$ with $n = n_1 + ... + n_R$.

$$
    A = 
    \left[
    \begin{array}{ccc}
    e^{T} & & \\
     & \ddots & \\
     & & e^{T}\\
    A_{1} & \cdots & A_{R}\\
    \end{array}
    \right]
$$
where $e^{T} = (1, ..., 1)$.

In [3]:
# Dense block-angular matrix
# where matrices A_1, ..., A_R are stored as dense matrices
struct DenseBlockAngular
    m::Int  # number of linking constraints
    n::Int  # number of columns (total)
    
    R::Int  # number of blocks
    ncols::Array{Int64, 1}  # number of columns in each block [n_1, ..., n_{R}]

    cols::Array{Array{Float64,2},1}  # columns [A_1, ..., A_R]
end

## Cholesky factors

Normal equations write $A \Theta A^{T} y = \xi$, i.e., when expanding the block-structure:

$$
    \left[
    \begin{array}{ccc|c}
    e^{T} \Theta_{1} e & & & e^{T} \Theta_{1} A_{1}^{T}\\
    & \ddots && \vdots\\
    && e^{T} \Theta_{R} e & e^{T} \Theta_{R} A_{R}^{T}\\
    \hline
    A_{1} \Theta_{1} e & \cdots & A_{1} \Theta_{1} e & \sum_{r} A_{r} \Theta_{r} A_{r}^{T}
    \end{array}
    \right]
    \left[
    \begin{array}{l}
    y_{1}\\
    \vdots\\
    y_{R}\\
    y_{0}
    \end{array}
    \right]
    =
    \left[
     \begin{array}{l}
    \xi_{1}\\
    \vdots\\
    \xi_{R}\\
    \xi_{0}
    \end{array}
    \right]
$$

Let $\Phi$ be the left-hand matrix of the above system.

The Cholesky factorization $\Phi = LDL^{T}$ is given by:

$$
    D = 
    \left[
    \begin{array}{cccc}
    d_{1} & & & \\
     & \ddots & &  \\
     & & d_{R}& \\
    &  &  & D_{C}\\
    \end{array}
    \right]
    \ \ 
    and \ \ 
    L =
    \left[
    \begin{array}{cccc}
    1 & & & \\
     & \ddots & & \\
     & & 1 & \\
    d_{1}^{-1} \eta_{1} & \cdots & d_{R}^{-1} \eta_{R} & L_{C}\\
    \end{array}
    \right]
$$
where:

\begin{align}
d_{r} &= e^{T} \Theta_{r} e\\
\eta_{r} &= A_{r} \Theta_{r} e\\
C &= \sum_{r} A_{r} \Theta_{r} A_{r}^{T} - \sum_{r} d_{r}^{-1} \eta_{r} \eta_{r}^{T}\\
&= L_{C} D_{C} L_{C}^{T}
\end{align}


In [4]:
# (Implicit) Cholesky factor of ADA' where A is primal block-angular
mutable struct FactorBlockAngular
    m::Int  # number of linking constraints
    n::Int  # number of columns (total)
    R::Int  # number of blocks
    ncols::Array{Int64, 1}  # number of columns in each block [n_1, ..., n_{R}]
    
    
    d::Array{Float64, 1}  # Rx1 vector
    eta::Array{Float64, 2}  # mxR matrix
    
#     # Schur complement
#     C::Array{Float64, 2}
    # Cholesky factor of (dense) Schur complement
    Fc::Base.LinAlg.Cholesky{Float64,Array{Float64,2}}
end

In [5]:
"""
    cholBlockAngular(A, F)
"""

function cholBlockAngular!(A::DenseBlockAngular, theta::Array{Float64,1}, F::FactorBlockAngular)
    
    # Schur complement
    C = zeros(A.m, A.m)
    
    colidx = 1
    for i=1:A.R
        theta_ = theta[colidx:colidx+A.ncols[i]-1]
        colidx += A.ncols[i]
        
        # diagonal elements
        F.d[i] = 1. / sum(theta_)
        
        # lower factors
        F.eta[:, i] = A.cols[i] * theta_
        
        # Schur complement
        CpBDBt!(C, A.cols[i], theta_)
        Base.BLAS.syr!('U', -F.d[i], F.eta[:, i], C)
    end
    
    # compute Cholesky factor of C
    F.Fc = cholfact(Symmetric(C))
        
    return nothing
end

function cholBlockAngular(A::DenseBlockAngular, theta::Array{Float64,1})
    
    # Schur complement
    C = zeros(A.m, A.m)
    d = zeros(A.R)
    eta = zeros(A.m, A.R)
    
    colidx = 1
    for i=1:A.R
        theta_ = theta[colidx:(colidx+A.ncols[i]-1)]
        colidx += A.ncols[i]
        
        # diagonal elements
        d[i] = 1. / sum(theta_)
        
        # lower factors
        eta[:, i] = A.cols[i] * theta_
        
        # Schur complement
        CpBDBt!(C, A.cols[i], theta_)
        Base.BLAS.syr!('U', -d[i], eta[:, i], C)
        
    end
    
    # compute Cholesky factor of C
    Fc = cholfact(Symmetric(C))
    
    # create implicit Cholesky factor
    F = FactorBlockAngular(A.m, A.n, A.R, A.ncols, d, eta, Fc)
    
    return F
end

cholBlockAngular (generic function with 1 method)

## Solving the normal equations

Once the Cholesky factorization is known, the normal equations are solved as follows:

\begin{align}
y_{0} &= C \setminus \left( \xi_{0} - \sum_{r} \xi_{r} d_{r}^{-1} \eta_{r} \right)\\
\forall r, \ \ \ y_{r} & = d_{r}^{-1}(\xi_{r} - \eta_{r}^{T}y_{0})
\end{align}

In [6]:
"""
    solvene!(F, b, y)

    Solve normal equations (A*D*A') y = b,
    where A is primal block-angular and F is the factor of (A*D*A')

    The solution is returned in `y`, which is modified in-place.
"""

function solvene!(F::FactorBlockAngular, b::Array{Float64, 1}, y::Array{Float64,1})
    
    # right-hand side for global solve (y0)
    rhs = zeros(F.m)
    for i=1:m
        rhs[i] = b[R+i]
    end
    
    # compute right-hand side
    for r=1:F.R
        Base.BLAS.axpy!(-F.d[r] * b[r], F.eta[:, r], rhs)
    end
    
    # compute y0
    y0 = F.Fc \ rhs
    for i=1:m
        y[R+i] = y0[i]
    end
    
    # local backward pass
    for r=1:R
        y[r] = F.d[r] * (b[r] - dot(F.eta[:, r], y0))
    end
    
    return nothing
end

solvene!

# Numerical tests

In [7]:
# Number of threads used b BLAS
Base.LinAlg.BLAS.set_num_threads(1)

In [8]:
function generate_instance(m, n, R, p, sp=false)
    
    if sp
        A_ = sprand(m, n*R, p);
    else
        A_ = rand(m, n*R);
    end

    # for the list of sparse matrices
    B_ = [A_[:, 1+n*(r-1):n*r] for r=1:R];
    
    e = kron(speye(R), sparse(ones(1, n)));
    A = vcat(e, A_);
    
    # diagonal matrix
    theta = rand(n*R);
    
    # right-hand side
    b = rand(R+m);
    
    return A, B_, theta, b
end

generate_instance (generic function with 2 methods)

In [9]:
m = 96
R = 8192
ncols = [128 for i=1:R]
n = sum(ncols)
println("Linking constraints: ", m)
println("Number of resources: ", R)
println("Columns (total)    : ", n)

Linking constraints: 96
Number of resources: 8192
Columns (total)    : 1048576


In [10]:
A, B_, theta, b = generate_instance(m, ncols[1], R, 0.0001, false);

In [11]:
# change conditionning to make it very bad
# If master problem is non-degenrate, there should be ~ m+R non-zero variables
for i=1:n
    if rand() < (m+R) / (n)
        theta[i] *= 10.0^8
    else
        theta[i] *= 10.0^-8
    end
end

## Naive implementation

In [12]:
@time S_ = Symmetric(A * spdiagm(theta)*A');

 16.165078 seconds (3.37 M allocations: 4.340 GiB, 0.92% gc time)


In [13]:
@time F_ = cholfact(S_);

  0.323088 seconds (78.28 k allocations: 121.579 MiB)


In [14]:
@time y_ = F_ \ b;

  0.057739 seconds (22.26 k allocations: 1.446 MiB)


## Specialized Cholesky

In [15]:
# create block-angular matrix
@time A_block = DenseBlockAngular(m, n, R, ncols, B_);

  0.003900 seconds (871 allocations: 50.121 KiB)


In [16]:
# compute Cholesky factorization
@time F = cholBlockAngular(A_block, theta);

  1.746110 seconds (167.24 k allocations: 35.652 MiB)


In [17]:
# solve normal equations
y = zeros(A_block.m+A_block.R)
@time solvene!(F, b, y)

  0.091991 seconds (272.62 k allocations: 20.930 MiB)


## Compare results

In [18]:
# distance between two results
maximum(abs.(y - y_))

1.3969838619232178e-9

In [19]:
# residual norm
println("Naive: ", maximum(abs.(S_*y_ - b)))
println("Spec : ", maximum(abs.(S_ * y - b)))

Naive: 1.8697853931826103e-8
Spec : 1.2746131483609702e-8
